## CODIGO


# Model Design

In [ ]:
# Model design
#pip install agentpy
import agentpy as ap
import numpy as np

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import IPython

# Aux
import random

## Model Definition

In [ ]:
class VehicleAgent(ap.Agent):

  """ An agent tha simulates a vehicle """
  
  def setup(self):
    self.map = self.model.map
    self.random = self.model.random
    self.steps = 0
    self.fuel = 0
    self.status = "looking"
    self.routeParking = None
    self.parkingCoordinates = None

  def moveUp(self):
    self.map.move_by(self, (-1, 0))

  def park(self):
    self.status = 'parked'

  def findParking(self):
    pAgents = [i for i in self.model.map.agents if 'ParkingAgent' in i.type and i.availability]

    if len(pAgents) > 0:
      bestParking = None
      path = [None] * 99999

      for parkA in pAgents:

        greenFlag = True

        neighbors = self.map.neighbors(self, distance=0)
        roadNeighbors = [i for i in neighbors if ('VehicleAgent' not in i.type) and ('ParkingAgent' not in i.type)]
        currentPos = self.map.positions[roadNeighbors[0]]
        tempPath = []
        parkingPosition = self.map.positions[parkA]

        while greenFlag and currentPos != parkingPosition:
          disToParking = abs(currentPos[0] - parkingPosition[0]) + abs(currentPos[1] - parkingPosition[1])
          
          if disToParking == 1:
            nextPos = parkingPosition

          elif len(roadNeighbors) == 1:
            if(roadNeighbors[0].direction == 'North'):
              nextPos = (currentPos[0]-1, currentPos[1])
            elif(roadNeighbors[0].direction == 'South'):
              nextPos = (currentPos[0]+1, currentPos[1])
            elif(roadNeighbors[0].direction == 'West'):
              nextPos = (currentPos[0], currentPos[1]-1) 
            else:
              nextPos = (currentPos[0], currentPos[1]+1)

          elif len(roadNeighbors) > 1:
            directions = [i.direction for i in roadNeighbors]

            if 'South' in directions and 'West' in directions:
              southValue = abs((currentPos[0] + 2) - parkingPosition[0]) + abs(currentPos[1] - parkingPosition[1])
              westValue = abs(currentPos[0] - parkingPosition[0]) + abs((currentPos[1] - 2) - parkingPosition[1])
              if westValue < southValue:
                nextPos = (currentPos[0], currentPos[1]-1)
              else:
                nextPos = (currentPos[0]+1, currentPos[1])

            elif 'North' in directions and 'West' in directions:
              northValue = abs((currentPos[0] - 2) - parkingPosition[0]) + abs(currentPos[1] - parkingPosition[1])
              westValue = abs(currentPos[0] - parkingPosition[0]) + abs((currentPos[1] - 2) - parkingPosition[1])
              if northValue < westValue:
                nextPos = (currentPos[0]-1, currentPos[1])
              else:
                nextPos = (currentPos[0], currentPos[1]-1)

            elif 'South' in directions and 'East' in directions:
              southValue = abs((currentPos[0] + 2) - parkingPosition[0]) + abs(currentPos[1] - parkingPosition[1])
              eastValue = abs(currentPos[0] - parkingPosition[0]) + abs((currentPos[1] + 2) - parkingPosition[1])
              if southValue < eastValue:
                nextPos = (currentPos[0]+1, currentPos[1])
              else:
                nextPos = (currentPos[0], currentPos[1]+1)

            elif 'North' in directions and 'East' in directions:
              northValue = abs((currentPos[0] - 2) - parkingPosition[0]) + abs(currentPos[1] - parkingPosition[1])
              eastValue = abs(currentPos[0] - parkingPosition[0]) + abs((currentPos[1] + 2) - parkingPosition[1])
              if eastValue < northValue:
                nextPos = (currentPos[0], currentPos[1]+1)
              else:
                nextPos = (currentPos[0]-1, currentPos[1])

          if nextPos[0] < 0 or nextPos[0] > self.map.shape[0]-1 or nextPos[1] < 0 or nextPos[1] > self.map.shape[1]-1 or len(tempPath) > 40 or len(tempPath) > len(path):
            greenFlag = False

          tempPath.append(nextPos)
          neighbors = self.map.neighbors(roadNeighbors[0], distance=1)
          roadNeighbors = [i for i in neighbors if ('VehicleAgent' not in i.type) and (self.map.positions[i] == nextPos)]
          currentPos = nextPos

        if greenFlag and len(tempPath) < len(path):
          path = tempPath
          bestParking = parkA
      
      if bestParking != None:
        # Reservar estacionamiento
        self.parkingCoordinates = self.map.positions[bestParking]
        self.routeParking = path
        self.status = 'found'
        bestParking.availability = False

    print("Id: " + str(self.id) + ", Vehículo: " + str(self.map.positions[self]) + ", ParkingCercano: " + str(self.parkingCoordinates))
    print(self.routeParking)
    print()

  def movement(self):
    # Obtenemos los vecinos al vehiculo
    neighbors = self.map.neighbors(self, distance = 0)
    roadNeighbors = [i for i in neighbors if 'VehicleAgent' not in i.type and 'ParkingAgent' not in i.type]

    # Observamos los posibles movimientos <- Aqui logica de mejor mov al estacionamiento
    # Movimiento cuando la celda va en un sólo sentido o cuando el carro está dejando el mapa porque no encontró un parking.
    # También funciona en los cruces, eligiendo una dirección aleatoria del cruce.
    if self.status == 'leaving' or self.status == 'looking':
      # Si llegamos al borde removemos al agente del Grid
      pos = self.map.positions[self]
      if self.steps > 0 and (pos[0] == 0 or pos[0] == self.map.shape[0]-1 or pos[1] == 0 or pos[1] == self.map.shape[1]-1):
        self.map.remove_agents(self)
      else:
        decision = random.randint(0, len(roadNeighbors)-1)

        if(roadNeighbors[decision].direction == 'North'):
          self.map.move_by(self, (-1, 0))

        elif(roadNeighbors[decision].direction == 'South'):
          self.map.move_by(self, (1, 0))

        elif(roadNeighbors[decision].direction == 'West'):
          self.map.move_by(self, (0, -1))
        
        else:
          self.map.move_by(self, (0, 1))
    
    # Movimiento cuando el auto se dirige hacia el estacionamiento que reservó.
    elif self.status == 'found':
      if self.routeParking:
        self.map.move_to(self, self.routeParking.pop(0))
      else:
        self.park()
    else:
      pass



In [ ]:
class ParkingAgent(ap.Agent):

  """ An agent that simulates a parking slot """

  def setup(self):
    self.map = self.model.map
    self.random = self.model.random
    self.id
    self.availability = True

  def setAvailability(self):
    # Condicional en dependencia a los agentes vehiculo
    pass

In [ ]:
def randomPos(n):
    positions = [(0,4), (11,0), (15, 11), (4, 15)]
    return random.sample(positions, n)


In [ ]:
class CongestionModel(ap.Model):

  """ A simple model of random wealth transfers """

  def setup(self):

    # Creacion de Grid
    self.map = ap.Grid(self, (16, 16), track_empty = True)

    # Creacion de Agentes
    self.vehicleAgents = []
    self.parkingAgents = ap.AgentList(self, self.p.parkingAgents, ParkingAgent)

    #parkPositions = [(3,1), (3,8), (3,14), (6,5), (6,10), (8,3), (10,9), (10,14), (12,2), (12,7)]
    parkPositions = [(0,10), (0,12), (1,3), (1,5), (1,10), (1,12), (2,3), (2,5), (2,10), (2,12), (3,0), (3,1), (3,2), (3,6), (3,7), (3,8), (3,9), (3,13), (3,14), (5,0), (5,1), (5,2), (5,6), (5,7), (5,8), (5,9), (5,13), (5,14), (6,3), (6,5), (6,10), (6,12), (7,3), (7,5), (7,10), (7,12), (8,3), (8,5), (8,10), (8,12), (9,3), (9,5), (9,10), (9,12), (10,1), (10,2), (10,6), (10,7), (10,8), (10,9), (10,13), (10,14), (10,15), (12,1), (12,2), (12,6), (12,7), (12,8), (12,9), (12,13), (12,14), (12,15), (13,3), (13,5), (13,10), (13,12), (14,3), (14,5), (14,10), (14,12), (15,3), (15,5)]
    
    parkingSet = random.sample(parkPositions, self.p.parkingAgents)

    #print(parkingSet)
    
    # Generacion de Agentes Carretera y posiciones
    road = []
    pos = []

    for i in range(0,4,1):
      
      temp = ap.AgentList(self, 16)
      
      if (i == 0):
        temp.direction = "East"
        for j in range(0,16,1):
          pos += [(11,j)]
      elif (i == 1):
        temp.direction = "West"
        for j in range(0,16,1):
          pos += [(4,j)]
      elif (i == 2):
        temp.direction = "South"
        for j in range(0,16,1):
          pos += [(j,4)]
      else:
        temp.direction = "North"
        for j in range(0,16,1):
          pos += [(j,11)]
      
      road += temp
    
    self.roads = road
    

    #self.map.add_agents(self.vehicleAgents, random=True, empty=True)
    
    self.map.add_agents(self.parkingAgents, positions = parkingSet)
    self.map.add_agents(self.roads, positions = pos)
    
 
  def update(self):
    if self.t % self.p.spawnIntensity == 0:

      #self.record('Traffic', traffic())
      numCars = random.randint(0,3)
      positions = randomPos(numCars)

      newCars = ap.AgentList(self, numCars, VehicleAgent)

      self.vehicleAgents += newCars

      if(len(positions) > 0):
        self.map.add_agents(newCars, positions = (positions))
        newCars.findParking()

  def step(self):
    vehAgents = [i for i in self.map.agents if 'VehicleAgent' in i.type]

    for vehicle in vehAgents:
      vehicle.movement()
      vehicle.steps += 1

    #print(positions)


  def end(self):
    pass
    #self.agents.record('')

## Simulation Run

In [ ]:
parameters = {
  #'vehicleAgents': 10,
  'parkingAgents': 19,
  'spawnIntensity': 5,
  'steps': 100,
  'seed': 42,
}

model = CongestionModel(parameters)
#results = model.run()
#results

In [ ]:
# Create single-run animation with custom colors

def view_agents(Agent):
  num = 0

  if('VehicleAgent' in Agent.type):
    num = 0
  elif('ParkingAgent' in Agent.type):
    num = 1
  elif('Agent' in Agent.type):
    num = 2
  else:
    num = 3

  return num
# leaving (ya se va de su estacionamiento, o nunca buscó estacionamiento)
# looking (cuando busca estacionamiento)
def animation_plot(model, ax):
    attr_grid = model.map.apply(view_agents)
    #print(attr_grid)
    #attr_grid = model.map.attr_grid(isinstance())
    color_dict = {0:'#7FC97F', 1:'#d62c2c', 2:'#848484', 3:'#e5e5e5', None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    #ax.set_title(f"Simulation of a forest fire\n"
                 #f"Time-step: {model.t}, Trees left: "
                 #f"{len(model.agents.select(model.agents.condition == 0))}")

fig, ax = plt.subplots()
model = CongestionModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=10))